In [ ]:
!pip install openai

In [ ]:
!pip install langchain langchain-community

In [ ]:
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain

In [ ]:
OPENAI_API_KEY = '...'

In [ ]:
!pip install youtube-transcript-api
!pip install pytube

### 1. Simple Videos

In [ ]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=AXq0QHUwmh8", add_video_info=True)

In [ ]:
result = loader.load()

In [ ]:
print (type(result))
print (f"Found video from {result[0].metadata['author']} that is {result[0].metadata['length']} seconds long")
print ("")
print (result)

In [ ]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

### 2. Long Videos

In [ ]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=6Ub7Z1AGIuk", add_video_info=True)

In [ ]:
result = loader.load()

**This below code will give an error. This is to show that document is long and we have to use Text Splitter. Continue with the next cell**

In [ ]:
chain = load_summarize_chain(llm, chain_type="stuff", verbose=False)
chain.run(result)

**Problem, your transcript/document is too long**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

In [ ]:
texts = text_splitter.split_documents(result)

In [ ]:
!pip install tiktoken

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(texts[:4])

### 3. Multiple Videos

In [ ]:
youtube_url_list = ["https://www.youtube.com/watch?v=AXq0QHUwmh8", "https://www.youtube.com/watch?v=EwHrjZxAT7g"]

texts = []

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)

for url in youtube_url_list:
    loader = YoutubeLoader.from_youtube_url(url, add_video_info=True)
    result = loader.load()

    texts.extend(text_splitter.split_documents(result))

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=False)
chain.run(texts)